<a href="https://colab.research.google.com/github/aghapygad336/MENA/blob/master/gtd_happiness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import random
import pandas as pd
import numpy as np
from openpyxl.utils import dataframe
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import seaborn as sns
from math import sqrt
import matplotlib.pyplot as plt
import math
from matplotlib import colors as cs
from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale
from sklearn import preprocessing, metrics
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, precision_score, recall_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import GridSearchCV

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np

data2015 = pd.read_csv('/content/drive/My Drive/MENA/2015.csv',encoding = "ISO-8859-1", engine = 'python')
data2016 = pd.read_csv('/content/drive/My Drive/MENA/2016.csv',encoding = "ISO-8859-1", engine = 'python')
data2017 = pd.read_csv('/content/drive/My Drive/MENA/2017.csv',encoding = "ISO-8859-1", engine = 'python')
happiness = data2015.append(data2016.append(data2017,sort=True),sort= True)

gtd = pd.read_csv('/content/drive/My Drive/MENA/golbal.csv',encoding = "ISO-8859-1", engine = 'python', usecols=[1,2,3,8,10,19,20,21,22,26,27,29,35,41,71,84,100,101,103,58])
gtd = gtd[(gtd['region_txt'] =='Middle East & North Africa') & (gtd['iyear'] > 2000)]


In [0]:
# gtd missing values
numeric_ = gtd._get_numeric_data()
cat = gtd.select_dtypes(include='object')

mean_nan = numeric_.fillna(numeric_.mean()).dropna(axis=1, how='all')
cat_nan = cat.loc[:, cat.isnull().mean() <0.25]
gtd = mean_nan.join(cat_nan)


In [0]:
# happiness missing values
numeric_ = happiness._get_numeric_data()
cat = happiness.select_dtypes(include='object')

mean_nan = numeric_.fillna(numeric_.mean()).dropna(axis=1, how='all')
cat_nan = cat.loc[:, cat.isnull().mean() <0.25]
happiness = mean_nan.join(cat_nan)

In [0]:
def encoding(dfData):    
    
    df = dfData.copy().select_dtypes(include='object')
    # print(df.columns)
    rows,cols = dfData.shape
    encodings = {}
    for col in df.columns:
        Region_nums = []
        Region_nums = df[col].unique()
        Region_nums = { Region_nums[i]: i for i in range(len(Region_nums)) }
        encodings.update({col:Region_nums})
        dfData[col].replace(Region_nums, inplace=True)
    return dfData,encodings

def get_by_value(dic,value):
    for key, val in dic.items():
        if val == value :
            return key    

In [0]:
gtd, gtd_encodings = encoding(gtd)
happiness, happiness_encodings = encoding(happiness)


In [37]:
get_by_value(gtd_encodings['gname'],67)

'Al-Qaida in the Arabian Peninsula (AQAP)'

In [38]:
top_5 = gtd['gname'].value_counts()[1:6]
top_5 = [203,27,107,67,58]
gtd = gtd[gtd['gname'].isin(top_5)]
gtd

,iyear,imonth,iday,crit1,crit2,crit3,doubtterr,success,suicide,claimed,nkillter,nwound,nwoundte,country_txt,region_txt,attacktype1_txt,targtype1_txt,natlty1_txt,gname,weapsubtype1_txt
72138,2001,4,4,1,1,1,0.0,1,0,0.0,0.0,2.00000,0.000000,7,0,0,7,10,27,15
75727,2003,7,8,1,1,1,0.0,0,0,0.0,0.0,1.00000,0.049719,1,0,3,7,1,27,5
75737,2003,7,10,1,1,1,1.0,1,0,0.0,0.0,1.00000,0.000000,1,0,4,1,1,27,4
75948,2003,10,12,1,1,1,0.0,1,1,1.0,1.0,35.00000,0.000000,7,0,0,5,10,58,6
76090,2003,12,1,1,1,1,0.0,1,0,0.0,0.0,4.00000,0.000000,1,0,0,8,1,27,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181639,2017,12,29,1,1,1,0.0,1,0,0.0,0.0,4.89981,0.000000,7,0,1,1,10,203,2
181670,2017,12,31,1,1,1,0.0,1,0,1.0,0.0,0.00000,0.000000,7,0,1,3,10,203,2
181671,2017,12,31,1,1,1,0.0,1,0,0.0,0.0,9.00000,0.000000,7,0,0,1,10,203,0
181674,2017,12,31,1,1,0,1.0,0,0,0.0,5.0,0.00000,0.000000,7,0,1,8,10,203,2


In [39]:
happiness

,Dystopia Residual,Dystopia.Residual,Economy (GDP per Capita),Economy..GDP.per.Capita.,Family,Freedom,Generosity,Happiness Rank,Happiness Score,Happiness.Rank,Happiness.Score,Health (Life Expectancy),Health..Life.Expectancy.,Lower Confidence Interval,Standard Error,Trust (Government Corruption),Trust..Government.Corruption.,Upper Confidence Interval,Whisker.high,Whisker.low,Country
0,2.51738,1.850238,1.39651,0.984718,1.34951,0.66557,0.29678,1.0,7.587,78.0,5.354019,0.94143,0.551341,5.282395,0.034110,0.41978,0.12312,5.481975,5.452326,5.255713,0
0,2.51738,1.850238,1.39651,0.984718,1.34951,0.66557,0.29678,1.0,7.587,78.0,5.354019,0.94143,0.551341,5.282395,0.034110,0.41978,0.12312,5.481975,5.452326,5.255713,1
0,2.51738,1.850238,1.39651,0.984718,1.34951,0.66557,0.29678,1.0,7.587,78.0,5.354019,0.94143,0.551341,5.282395,0.034110,0.41978,0.12312,5.481975,5.452326,5.255713,2
0,2.73939,1.850238,1.44178,0.984718,1.16374,0.57941,0.36171,1.0,7.526,78.0,5.354019,0.79504,0.551341,7.460000,0.047885,0.44453,0.12312,7.592000,5.452326,5.255713,0
0,2.73939,1.850238,1.44178,0.984718,1.16374,0.57941,0.36171,1.0,7.526,78.0,5.354019,0.79504,0.551341,7.460000,0.047885,0.44453,0.12312,7.592000,5.452326,5.255713,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,1.83302,1.850238,0.01530,0.984718,0.41587,0.11850,0.19727,157.0,2.905,78.0,5.354019,0.22396,0.551341,5.282395,0.086580,0.10062,0.12312,5.481975,5.452326,5.255713,165
156,1.83302,1.850238,0.01530,0.984718,0.41587,0.11850,0.19727,157.0,2.905,78.0,5.354019,0.22396,0.551341,5.282395,0.086580,0.10062,0.12312,5.481975,5.452326,5.255713,165
156,2.10404,1.850238,0.06831,0.984718,0.23442,0.04320,0.20290,157.0,2.905,78.0,5.354019,0.15747,0.551341,2.732000,0.047885,0.09419,0.12312,3.078000,5.452326,5.255713,165
156,2.10404,1.850238,0.06831,0.984718,0.23442,0.04320,0.20290,157.0,2.905,78.0,5.354019,0.15747,0.551341,2.732000,0.047885,0.09419,0.12312,3.078000,5.452326,5.255713,165


In [0]:

combinedData=pd.merge(gtd, happiness, left_on='country_txt', right_on='Country')

happiness, happiness_encodings = encoding(happiness)
combinedDataE, combinedData_encodings = encoding(combinedData)


In [41]:
combinedDataE.isna().sum()

iyear                            0
imonth                           0
iday                             0
crit1                            0
crit2                            0
crit3                            0
doubtterr                        0
success                          0
suicide                          0
claimed                          0
nkillter                         0
nwound                           0
nwoundte                         0
country_txt                      0
region_txt                       0
attacktype1_txt                  0
targtype1_txt                    0
natlty1_txt                      0
gname                            0
weapsubtype1_txt                 0
Dystopia Residual                0
Dystopia.Residual                0
Economy (GDP per Capita)         0
Economy..GDP.per.Capita.         0
Family                           0
Freedom                          0
Generosity                       0
Happiness Rank                   0
Happiness Score     

In [42]:
combinedDataE

,iyear,imonth,iday,crit1,crit2,crit3,doubtterr,success,suicide,claimed,nkillter,nwound,nwoundte,country_txt,region_txt,attacktype1_txt,targtype1_txt,natlty1_txt,gname,weapsubtype1_txt,Dystopia Residual,Dystopia.Residual,Economy (GDP per Capita),Economy..GDP.per.Capita.,Family,Freedom,Generosity,Happiness Rank,Happiness Score,Happiness.Rank,Happiness.Score,Health (Life Expectancy),Health..Life.Expectancy.,Lower Confidence Interval,Standard Error,Trust (Government Corruption),Trust..Government.Corruption.,Upper Confidence Interval,Whisker.high,Whisker.low,Country
0,2001,4,4,1,1,1,0.0,1,0,0.0,0.0,2.0,0.0,7,0,0,7,10,27,15,2.371190,1.850238,1.331710,0.984718,1.289070,0.659800,0.362620,8.000000,7.364000,78.0,5.354019,0.910870,0.551341,5.282395,0.031570,0.438440,0.123120,5.481975,5.452326,5.255713,7
1,2001,4,4,1,1,1,0.0,1,0,0.0,0.0,2.0,0.0,7,0,0,7,10,27,15,2.475530,1.850238,1.360660,0.984718,1.172780,0.581470,0.494010,8.000000,7.334000,78.0,5.354019,0.830960,0.551341,7.264000,0.047885,0.419040,0.123120,7.404000,5.452326,5.255713,7
2,2001,4,4,1,1,1,0.0,1,0,0.0,0.0,2.0,0.0,7,0,0,7,10,27,15,2.212032,2.046456,0.899837,1.405706,1.548195,0.614062,0.500005,79.238095,5.378949,8.0,7.314000,0.594054,0.816760,5.282395,0.047885,0.140532,0.382817,5.481975,7.379510,7.248490,7
3,2001,4,4,1,1,1,0.0,1,0,0.0,0.0,2.0,0.0,7,0,0,7,10,27,15,2.264250,1.850238,1.250180,0.984718,1.319670,0.639380,0.475010,9.000000,7.286000,78.0,5.354019,0.908370,0.551341,5.282395,0.033710,0.429220,0.123120,5.481975,5.452326,5.255713,7
4,2001,4,4,1,1,1,0.0,1,0,0.0,0.0,2.0,0.0,7,0,0,7,10,27,15,2.546500,1.850238,1.444430,0.984718,1.104760,0.568370,0.474070,9.000000,7.313000,78.0,5.354019,0.851200,0.551341,7.241000,0.047885,0.323310,0.123120,7.385000,5.452326,5.255713,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85477,2015,10,17,1,1,1,0.0,1,0,1.0,0.0,0.0,0.0,14,0,2,0,49,203,5,3.559060,1.850238,1.115080,0.984718,0.714600,0.377090,0.117350,21.000000,6.778000,78.0,5.354019,0.711430,0.551341,6.680000,0.047885,0.183550,0.123120,6.876000,5.452326,5.255713,14
85478,2015,10,17,1,1,1,0.0,1,0,1.0,0.0,0.0,0.0,14,0,2,0,49,203,5,2.212032,1.734704,0.899837,1.626343,1.266410,0.608345,0.360942,79.238095,5.378949,21.0,6.648000,0.594054,0.726798,5.282395,0.047885,0.140532,0.324490,5.481975,6.722047,6.573952,14
85479,2015,10,17,1,1,1,0.0,1,0,1.0,0.0,0.0,0.0,14,0,2,0,49,203,5,2.848480,1.850238,1.063530,0.984718,1.198500,0.542100,0.244340,25.000000,6.786000,78.0,5.354019,0.796610,0.551341,5.282395,0.049100,0.092700,0.123120,5.481975,5.452326,5.255713,14
85480,2015,10,17,1,1,1,0.0,1,0,1.0,0.0,0.0,0.0,14,0,2,0,49,203,5,3.005590,1.850238,1.183060,0.984718,0.989120,0.489270,0.241800,25.000000,6.701000,78.0,5.354019,0.708350,0.551341,6.601000,0.047885,0.084230,0.123120,6.801000,5.452326,5.255713,14


In [0]:
def SplitTrainTes(dataSet):
    train_dataSet=dataSet[(dataSet['iyear'] >=2000) & (dataSet['iyear'] <= 2016)]
    test_dataSet=dataSet[dataSet['iyear'] ==2017]
    return train_dataSet,test_dataSet

In [0]:
train_GTD,test_GTD=SplitTrainTes(gtd)
train_Combined,test_Combined=SplitTrainTes(combinedData)

In [0]:
def splitX_SplitY(trainD,testD):
    x_train = trainD.drop("gname", axis=1)
    y_train = trainD["gname"]
    x_test = testD.drop("gname", axis=1)
    y_test = testD["gname"]
    return x_train,y_train,x_test,y_test

In [0]:
x_trainGTD,y_trainGTD,x_testGTD,y_testGTD=splitX_SplitY(train_GTD,test_GTD)
x_trainCombined,y_trainCombined,x_testCombined,y_testCombined=splitX_SplitY(train_Combined,test_Combined)


In [0]:
from sklearn.metrics import multilabel_confusion_matrix
def Fmeasure(estimator,x_train,y_train,x_test,y_test):
    y_pred_train = estimator.predict(x_train)
    y_pred_test = estimator.predict(x_test)
    print('Train Set Accuracy : ', accuracy_score(y_train, y_pred_train))
    print('Train Set Precision : ', precision_score(y_train, y_pred_train, average='micro'))
    print('Train Set Recall : ', recall_score(y_train, y_pred_train, average='micro'))
    print('Train F-Score for each class : ', f1_score(y_train, y_pred_train, average='micro'))
    print('Train Mean F-Score for both classes : ', f1_score(y_train, y_pred_train, average='micro'))

    print('----------------------------------------------------------------------')
    print('Test Set Accuracy : ', accuracy_score(y_test, y_pred_test))
    print('Test Set Precision : ', precision_score(y_test, y_pred_test, average='micro'))
    print('Test Set Recall : ', recall_score(y_test, y_pred_test, average='micro'))
    print('Test F-Score for each class : ', f1_score(y_test, y_pred_test, average='micro'))
    print('Test Mean F-Score for both classes : ', f1_score(y_test, y_pred_test, average='micro'))

    print('----------------------------------------------------------------------')

In [0]:
def kNearestNeighborsFunction(x_train,y_train,x_test,y_test):
    neighbors = list(range(1, 30, 2))
    # empty list that will hold cv scores
    cv_scores = []
    # perform 10-fold cross validation
    for k in neighbors:
        knn = KNeighborsClassifier(n_neighbors=k)
        scores = cross_val_score(knn, x_train, y_train, cv=10, scoring='accuracy')
        cv_scores.append(scores.mean())

    mse = [1 - x for x in cv_scores]
    optimal_k = neighbors[mse.index(min(mse))]
    n=optimal_k
    print('Best N found at ' , n)
    knn = KNeighborsClassifier(n_neighbors=n)
    knn.fit(x_train,y_train)
    Fmeasure(knn,x_train,y_train,x_test,y_test)

In [0]:

kNearestNeighborsFunction(x_trainGTD,y_trainGTD,x_testGTD,y_testGTD)

In [0]:
kNearestNeighborsFunction(x_trainCombined,y_trainCombined,x_testCombined,y_testCombined)


In [0]:

def DT (x_train,y_train,x_test,y_test):
    ct_gini = DecisionTreeClassifier()
    ct_entropy = DecisionTreeClassifier(criterion="entropy")
    ct_gini = ct_gini.fit(x_train, y_train)
    ct_entropy = ct_entropy.fit(x_train, y_train)
    Fmeasure(ct_entropy,x_train,y_train,x_test,y_test)
    Fmeasure(ct_gini,x_train,y_train,x_test,y_test)

In [0]:
DT(x_trainGTD,y_trainGTD,x_testGTD,y_testGTD)

In [0]:
DT(x_trainCombined,y_trainCombined,x_testCombined,y_testCombined)


In [0]:
def NB(x_train,y_train,x_test,y_test):
    nb = GaussianNB()
    nb = nb.fit(x_train, y_train)
    Fmeasure(nb,x_train,y_train,x_test,y_test)


In [0]:
NB(x_trainGTD,y_trainGTD,x_testGTD,y_testGTD)

In [0]:
NB(x_trainCombined,y_trainCombined,x_testCombined,y_testCombined)


In [0]:
def LR(x_train,y_train,x_test,y_test):
    import warnings
    warnings.filterwarnings("ignore")

    grid = {"C": np.logspace(-3, 3, 7), "penalty": ["l1", "l2"]}  # l1 lasso l2 ridge
    logreg = LogisticRegression()
    logreg_cv = GridSearchCV(logreg, grid, cv=10)
    logreg_cv.fit(x_train, y_train)
    print("tuned hpyerparameters :(best parameters) ", logreg_cv.best_params_)
    Fmeasure(logreg_cv,x_train,y_train,x_test,y_test)
    print("accuracy :", logreg_cv.best_score_)

In [0]:
LR(x_trainGTD,y_trainGTD,x_testGTD,y_testGTD)

In [0]:
def AB(x_train,y_train,x_test,y_test):
    import warnings
    warnings.filterwarnings("ignore")

    param_grid = {
        'learning_rate': [.1, .2, .3, .4, .5],

        'n_estimators': [50, 100, 150, 200, 250]
    }

    classifier = AdaBoostClassifier()
    grid_Search = GridSearchCV(classifier, param_grid=param_grid)
    kk = grid_Search.fit(x_train, y_train)
    Fmeasure(kk,x_train,y_train,x_test,y_test)

In [0]:
AB(x_trainGTD,y_trainGTD,x_testGTD,y_testGTD)

In [0]:
AB(x_trainCombined,y_trainCombined,x_testCombined,y_testCombined)


In [0]:
def SVM(x_train,y_train,x_test,y_test):
    from sklearn.svm import SVC
    svclassifier = SVC(kernel='poly', degree=8)
    svclassifier.fit(x_train, y_train)
    Fmeasure(svclassifier,x_train,y_train,x_test,y_test)



In [0]:
SVM(x_trainGTD,y_trainGTD,x_testGTD,y_testGTD)